<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Setup---Debug" data-toc-modified-id="Setup---Debug-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Setup - Debug</a></span></li><li><span><a href="#Setup---Imports" data-toc-modified-id="Setup---Imports-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Setup - Imports</a></span></li><li><span><a href="#Setup---working-folder-paths" data-toc-modified-id="Setup---working-folder-paths-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Setup - working folder paths</a></span></li><li><span><a href="#Setup---logging" data-toc-modified-id="Setup---logging-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Setup - logging</a></span></li><li><span><a href="#Setup---virtualenv-jupyter-kernel" data-toc-modified-id="Setup---virtualenv-jupyter-kernel-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Setup - virtualenv jupyter kernel</a></span></li><li><span><a href="#Setup---Initialize-Django" data-toc-modified-id="Setup---Initialize-Django-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Setup - Initialize Django</a></span></li><li><span><a href="#Setup---Initialize-LoggingHelper" data-toc-modified-id="Setup---Initialize-LoggingHelper-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Setup - Initialize LoggingHelper</a></span></li></ul></li><li><span><a href="#Find-articles-to-be-loaded" data-toc-modified-id="Find-articles-to-be-loaded-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Find articles to be loaded</a></span><ul class="toc-item"><li><span><a href="#Uncompress-files" data-toc-modified-id="Uncompress-files-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Uncompress files</a></span></li><li><span><a href="#Work-with-uncompressed-files" data-toc-modified-id="Work-with-uncompressed-files-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Work with uncompressed files</a></span></li><li><span><a href="#load-a-file-into-memory." data-toc-modified-id="load-a-file-into-memory.-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>load a file into memory.</a></span></li><li><span><a href="#build-list-of-all-ObjectTypes" data-toc-modified-id="build-list-of-all-ObjectTypes-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>build list of all ObjectTypes</a></span></li></ul></li><li><span><a href="#TODO" data-toc-modified-id="TODO-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>TODO</a></span></li></ul></div>

# Introduction

- Back to [Table of Contents](#Table-of-Contents)

This is a notebook that expands on the OpenCalais code in the file `article_coding.py`, also in this folder.  It includes more sections on selecting publications you want to submit to OpenCalais as an example.  It is intended to be copied and re-used.

# Setup

- Back to [Table of Contents](#Table-of-Contents)

## Setup - Debug

- Back to [Table of Contents](#Table-of-Contents)

In [1]:
debug_flag = False

## Setup - Imports

- Back to [Table of Contents](#Table-of-Contents)

In [2]:
import datetime
import glob
import logging
import lxml
import os
import six
import xml
import xmltodict
import zipfile

## Setup - working folder paths

- Back to [Table of Contents](#Table-of-Contents)

What data are we looking at?

In [3]:
# paper identifier
paper_identifier = "ChristianScienceMonitor"
archive_identifier = None

# source
source_paper_folder = "/mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data"
source_paper_path = "{}/{}".format( source_paper_folder, paper_identifier )
source_archive_file = "{}.zip".format( archive_identifier )
source_archive_path = "{}/{}".format( source_paper_path, source_archive_file )

# uncompressed
uncompressed_paper_folder = "/mnt/hgfs/projects/phd/proquest_hnp/uncompressed"
uncompressed_paper_path = "{}/{}".format( uncompressed_paper_folder, paper_identifier )

# make sure an identifier is set before you make a path here.
if ( ( archive_identifier is not None ) and ( archive_identifier != "" ) ):
    
    # identifier is set.
    uncompressed_archive_path = "{}/{}".format( uncompressed_paper_path, archive_identifier )

#-- END check to see if archive_identifier present. --#

## Setup - logging

- Back to [Table of Contents](#Table-of-Contents)

configure logging for this notebook's kernel (If you do not run this cell, you'll get the django application's logging configuration.

In [4]:
logging_file_name = "/home/jonathanmorgan/logs/django-research-data_load-{}.log.txt".format( paper_identifier )
logging.basicConfig(
    level = logging.DEBUG,
    format = '%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    filename = logging_file_name,
    filemode = 'w' # set to 'a' if you want to append, rather than overwrite each time.
)

## Setup - virtualenv jupyter kernel

- Back to [Table of Contents](#Table-of-Contents)

If you are using a virtualenv, make sure that you:

- have installed your virtualenv as a kernel.
- choose the kernel for your virtualenv as the kernel for your notebook (Kernel --> Change kernel).

Since I use a virtualenv, need to get that activated somehow inside this notebook.  One option is to run `../dev/wsgi.py` in this notebook, to configure the python environment manually as if you had activated the `sourcenet` virtualenv.  To do this, you'd make a code cell that contains:

    %run ../dev/wsgi.py
    
This is sketchy, however, because of the changes it makes to your Python environment within the context of whatever your current kernel is.  I'd worry about collisions with the actual Python 3 kernel.  Better, one can install their virtualenv as a separate kernel.  Steps:

- activate your virtualenv:

        workon research

- in your virtualenv, install the package `ipykernel`.

        pip install ipykernel

- use the ipykernel python program to install the current environment as a kernel:

        python -m ipykernel install --user --name <env_name> --display-name "<display_name>"
        
    `sourcenet` example:
    
        python -m ipykernel install --user --name sourcenet --display-name "research (Python 3)"
        
More details: [http://ipython.readthedocs.io/en/stable/install/kernel_install.html](http://ipython.readthedocs.io/en/stable/install/kernel_install.html)

## Setup - Initialize Django

- Back to [Table of Contents](#Table-of-Contents)

First, initialize my dev django project, so I can run code in this notebook that references my django models and can talk to the database using my project's settings.

In [5]:
# init django
django_init_folder = "/home/jonathanmorgan/work/django/research/work/phd_work"
django_init_path = "django_init.py"
if( ( django_init_folder is not None ) and ( django_init_folder != "" ) ):
    
    # add folder to front of path.
    django_init_path = "{}/{}".format( django_init_folder, django_init_path )
    
#-- END check to see if django_init folder. --#

In [6]:
%run $django_init_path

django initialized at 2019-08-10 02:03:22.746578


In [7]:
# context_text imports
from context_text.article_coding.article_coding import ArticleCoder
from context_text.article_coding.article_coding import ArticleCoding
from context_text.article_coding.open_calais_v2.open_calais_v2_article_coder import OpenCalaisV2ArticleCoder
from context_text.collectors.newsbank.newspapers.GRPB import GRPB
from context_text.collectors.newsbank.newspapers.DTNB import DTNB
from context_text.models import Article
from context_text.models import Article_Subject
from context_text.models import Newspaper
from context_text.shared.context_text_base import ContextTextBase


## Setup - Initialize LoggingHelper

- Back to [Table of Contents](#Table-of-Contents)

Create a LoggingHelper instance to use to log debug and also print at the same time.

Preconditions: Must be run after Django is initialized, since `python_utilities` is in the django path.

In [8]:
# python_utilities
from python_utilities.logging.logging_helper import LoggingHelper

# init
my_logging_helper = LoggingHelper()
my_logging_helper.set_logger_name( "proquest_hnp-article-loading-{}".format( paper_identifier ) )
log_message = None

# Find articles to be loaded

- Back to [Table of Contents](#Table-of-Contents)

Specify which folder of XML files should be loaded into system, then process all files within the folder.

The compressed archives from proquest_hnp just contain publication XML files, no containing folder.

To process:

- **uncompresed paper folder ( `<paper_folder>` )** - make a folder in `/mnt/hgfs/projects/phd/proquest_hnp/uncompressed` for the paper whose data you are working with, named the same as the paper's folder in `/mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data`.

    - for example, for the Boston Globe, name it "`BostonGlobe`".

- **uncompressed archive folder ( `<archive_folder>` )** - inside a given paper's folder in uncompressed, for each archive file, create a folder named the same as the archive file, but with no ".zip" at the end.

    - For example, for the file "`BG_20171002210239_00001.zip`", make a folder named "`BG_20171002210239_00001`".
    - path should be "`<paper_folder>/<archive_name_no_zip>`.

- unzip the archive into this folder:

        unzip <path_to_zip> -d <archive_folder>



## Uncompress files

- Back to [Table of Contents](#Table-of-Contents)

See if the uncompressed paper folder exists.  If not, set flag and create it.

In [9]:
# declare variables
did_uncomp_paper_folder_exist = False

# check if uncompressed paper folder exists.
if not os.path.exists( uncompressed_paper_path ):
    
    # no.  Make it.
    os.makedirs( uncompressed_paper_path )
    did_uncomp_paper_folder_exist = False
    log_message = "CREATED - Uncompressed paper folder {}".format( uncompressed_paper_path )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )
    
else:
    
    # yes.  Set flag.
    did_uncomp_paper_folder_exist = True
    log_message = "EXISTS - Uncompressed paper folder {}".format( uncompressed_paper_path )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )
    
#-- END check to see if paper folder exists.

EXISTS - Uncompressed paper folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor


For each *.zip file in the paper's source folder:

- parse file name from path returned by glob.
- parse the part before ".zip" from the file name.  This is referred to subsequently as the "archive identifier".
- check if folder named the same as the "archive identifier" is present.

    - If no:
    
        - create it.
        - then, uncompress the archive into it.
        
    - If yes:
    
        - output a message.  Don't want to uncompress if it was already uncompressed once.

In [10]:
# declare variables - papers
did_uncomp_paper_folder_exist = None

# declare variables archive (.zip) files.
zip_file_list = None
zip_file_path = None
zip_file_path_parts_list = None
zip_file_name = None
zip_file_name_parts_list = None
archive_identifier = None
uc_archive_folder_path = None
zip_file = None

# declare variables - auditing (uc = uncompressed)
archive_file_counter = None
did_uc_archive_folder_exist = None
uc_folder_exists_counter = None
start_dt = None
end_dt = None
time_delta = None

# use glob to get list of zip files in paper source folder.
zip_file_list = glob.glob( "{}/*.zip".format( source_paper_path ) )
zip_file_count = len( zip_file_list )

log_message = "==> zip file count: {}".format( zip_file_count )
my_logging_helper.output_debug_message( log_message, do_print_IN = True )

# loop over zip files.
archive_file_counter = 0
did_uc_archive_folder_exist = False
uc_folder_exists_counter = 0
for zip_file_path in zip_file_list:
    
    # increment counter
    archive_file_counter += 1
    
    log_message = "----> processing file {} of {}".format( archive_file_counter, zip_file_count )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )
    
    # get file name
    
    # split path into parts on path separator.
    zip_file_path_parts_list = zip_file_path.split( "/" )

    # file name is the last thing in the list.
    zip_file_name = zip_file_path_parts_list[ -1 ]

    # archive_identifier is name with ".zip" removed from end.
    zip_file_name_parts_list = zip_file_name.split( ".zip" )
    archive_identifier = zip_file_name_parts_list[ 0 ]
    
    # for now, log and print the things we've just created.
    log_message = "==> path: {}".format( zip_file_path )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )
    log_message = "==> file: {}".format( zip_file_name )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )
    log_message = "==> ID: {}".format( archive_identifier )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )

    # check if uncompressed archive folder exists.
    uc_archive_folder_path = "{}/{}".format( uncompressed_paper_path, archive_identifier )

    log_message = "==> TO: {}".format( uc_archive_folder_path )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )

    # check if the uncompressed archive folder exists.
    did_uc_archive_folder_exist = os.path.exists( uc_archive_folder_path )
    if did_uc_archive_folder_exist == False:

        # no.  Make it.
        os.makedirs( uc_archive_folder_path )
        log_message = "CREATED - Uncompressed archive folder {}".format( uc_archive_folder_path )
        my_logging_helper.output_debug_message( log_message, do_print_IN = True )
        
        # and uncompress archive to it.
        with zipfile.ZipFile( zip_file_path, 'r' ) as zip_file:

            # starting extract.
            start_dt = datetime.datetime.now()
            log_message = "==> extract started at {}".format( start_dt )
            my_logging_helper.output_debug_message( log_message, do_print_IN = True )
            
            # unzip to uncompressed archive folder path.
            zip_file.extractall( uc_archive_folder_path )
            
            log_message = "EXTRACTED - {}".format( zip_file_path )
            my_logging_helper.output_debug_message( log_message, do_print_IN = True )

            log_message = "TO uncompressed archive folder - {}".format( uc_archive_folder_path )
            my_logging_helper.output_debug_message( log_message, do_print_IN = True )
            
            # complete
            end_dt = datetime.datetime.now()
            
            log_message = "==> extract completed at {}".format( end_dt )
            my_logging_helper.output_debug_message( log_message, do_print_IN = True )
            
            log_message = "==> time elapsed: {}".format( end_dt - start_dt )
            my_logging_helper.output_debug_message( log_message, do_print_IN = True )
            
        #-- END with ZipFile --#

    else:

        # yes.  Set flag.
        uc_folder_exists_counter += 1
        log_message = "EXISTS, so moving on - Uncompressed archive folder {}".format( uncompressed_archive_path )
        my_logging_helper.output_debug_message( log_message, do_print_IN = True )

    #-- END check to see if archive folder exists. --#

    log_message = "------------------------------"
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )

#-- END loop over zip files. --#

==> zip file count: 263
----> processing file 1 of 263
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413220638_00001.zip
==> file: CSM_20170413220638_00001.zip
==> ID: CSM_20170413220638_00001
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413220638_00001
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413220638_00001
==> extract started at 2019-08-09 21:43:56.938364
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413220638_00001.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413220638_00001
==> extract completed at 2019-08-09 21:44:48.515733
==> time elapsed: 0:00:51.577369
------------------------------
----> processing file 2 of 263
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Chr

==> extract started at 2019-08-09 21:52:35.327357
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413222417_00009.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413222417_00009
==> extract completed at 2019-08-09 21:53:34.819050
==> time elapsed: 0:00:59.491693
------------------------------
----> processing file 11 of 263
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413222518_00010.zip
==> file: CSM_20170413222518_00010.zip
==> ID: CSM_20170413222518_00010
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413222518_00010
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413222518_00010
==> extract started at 2019-08-09 21:53:35.284276
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Chris

==> extract started at 2019-08-09 22:02:10.189323
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413222729_00019.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413222729_00019
==> extract completed at 2019-08-09 22:03:04.454532
==> time elapsed: 0:00:54.265209
------------------------------
----> processing file 21 of 263
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413222830_00020.zip
==> file: CSM_20170413222830_00020.zip
==> ID: CSM_20170413222830_00020
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413222830_00020
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413222830_00020
==> extract started at 2019-08-09 22:03:04.948264
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Chris

==> extract started at 2019-08-09 22:12:01.178872
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413223955_00006.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413223955_00006
==> extract completed at 2019-08-09 22:12:57.053543
==> time elapsed: 0:00:55.874671
------------------------------
----> processing file 31 of 263
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413223956_00007.zip
==> file: CSM_20170413223956_00007.zip
==> ID: CSM_20170413223956_00007
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413223956_00007
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413223956_00007
==> extract started at 2019-08-09 22:12:57.453536
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Chris

==> extract started at 2019-08-09 22:21:15.542857
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413224209_00016.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413224209_00016
==> extract completed at 2019-08-09 22:22:10.395188
==> time elapsed: 0:00:54.852331
------------------------------
----> processing file 41 of 263
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413224211_00017.zip
==> file: CSM_20170413224211_00017.zip
==> ID: CSM_20170413224211_00017
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413224211_00017
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413224211_00017
==> extract started at 2019-08-09 22:22:10.668126
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Chris

...

==> extract started at 2019-08-10 00:54:56.409055
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413234115_00011.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413234115_00011
==> extract completed at 2019-08-10 00:55:50.707093
==> time elapsed: 0:00:54.298038
------------------------------
----> processing file 221 of 263
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413234116_00012.zip
==> file: CSM_20170413234116_00012.zip
==> ID: CSM_20170413234116_00012
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413234116_00012
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413234116_00012
==> extract started at 2019-08-10 00:55:51.004096
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Chri

==> extract started at 2019-08-10 01:03:49.598471
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413234428_00021.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413234428_00021
==> extract completed at 2019-08-10 01:04:43.050219
==> time elapsed: 0:00:53.451748
------------------------------
----> processing file 231 of 263
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413234429_00022.zip
==> file: CSM_20170413234429_00022.zip
==> ID: CSM_20170413234429_00022
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413234429_00022
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413234429_00022
==> extract started at 2019-08-10 01:04:43.348460
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Chri

==> extract started at 2019-08-10 01:12:33.186900
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413234740_00031.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413234740_00031
==> extract completed at 2019-08-10 01:13:24.869632
==> time elapsed: 0:00:51.682732
------------------------------
----> processing file 241 of 263
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413234741_00032.zip
==> file: CSM_20170413234741_00032.zip
==> ID: CSM_20170413234741_00032
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413234741_00032
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413234741_00032
==> extract started at 2019-08-10 01:13:25.161477
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Chri

==> extract started at 2019-08-10 01:21:28.318560
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413235053_00041.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413235053_00041
==> extract completed at 2019-08-10 01:22:20.655406
==> time elapsed: 0:00:52.336846
------------------------------
----> processing file 251 of 263
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413235054_00042.zip
==> file: CSM_20170413235054_00042.zip
==> ID: CSM_20170413235054_00042
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413235054_00042
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413235054_00042
==> extract started at 2019-08-10 01:22:20.916676
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Chri

==> extract started at 2019-08-10 01:30:01.895608
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413235305_00050.zip
TO uncompressed archive folder - /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413235305_00050
==> extract completed at 2019-08-10 01:30:55.583179
==> time elapsed: 0:00:53.687571
------------------------------
----> processing file 261 of 263
==> path: /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/ChristianScienceMonitor/CSM_20170413235306_00051.zip
==> file: CSM_20170413235306_00051.zip
==> ID: CSM_20170413235306_00051
==> TO: /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413235306_00051
CREATED - Uncompressed archive folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413235306_00051
==> extract started at 2019-08-10 01:30:55.855121
EXTRACTED - /mnt/hgfs/projects/phd/proquest_hnp/proquest_hnp/data/Chri

## Work with uncompressed files

- Back to [Table of Contents](#Table-of-Contents)

Change working directories to the uncompressed paper path.

In [11]:
%cd $uncompressed_paper_path

/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor


In [12]:
%ls

CSM_20170413220638_00001/  CSM_20170413225919_00021/  CSM_20170413232720_00015/
CSM_20170413222205_00001/  CSM_20170413225920_00022/  CSM_20170413232721_00016/
CSM_20170413222207_00002/  CSM_20170413225921_00023/  CSM_20170413232822_00017/
CSM_20170413222309_00003/  CSM_20170413225922_00024/  CSM_20170413232824_00018/
CSM_20170413222310_00004/  CSM_20170413230023_00025/  CSM_20170413232825_00019/
CSM_20170413222312_00005/  CSM_20170413230027_00026/  CSM_20170413232926_00020/
CSM_20170413222313_00006/  CSM_20170413230028_00027/  CSM_20170413232927_00021/
CSM_20170413222414_00007/  CSM_20170413230129_00028/  CSM_20170413232928_00022/
CSM_20170413222416_00008/  CSM_20170413230131_00029/  CSM_20170413232929_00023/
CSM_20170413222417_00009/  CSM_20170413230132_00030/  CSM_20170413233030_00024/
CSM_20170413222518_00010/  CSM_20170413230233_00031/  CSM_20170413233032_00025/
CSM_20170413222519_00011/  CSM_20170413230235_00032/  CSM_20170413233034_00026/
CSM_20170413222521_00012/  CSM_201704132

## load a file into memory.

- Back to [Table of Contents](#Table-of-Contents)

Load one of the files into memory and see what we can do with it.  Beautiful Soup?

Looks like the root element is "Record", then the high-level type of the article is "ObjectType".

ObjectType values:

- Advertisement
- ...

Good options for XML parser:

- `lxml.etree` - [https://stackoverflow.com/questions/12290091/reading-xml-file-and-fetching-its-attributes-value-in-python](https://stackoverflow.com/questions/12290091/reading-xml-file-and-fetching-its-attributes-value-in-python)
- `xmltodict` - [https://docs.python-guide.org/scenarios/xml/](https://docs.python-guide.org/scenarios/xml/)
- `beautifulsoup` using `lxml`

In [ ]:
# loop over files in the current archive folder path.

# declare variables
xml_file_list = None
xml_file_path = None
xml_file = None
xml_dict = None
xml_file_counter = None
object_type_to_count_map = None
object_type_count = None
record_node = None
object_type_node = None
object_type_list = None
object_type = None

# declare variables - auditing
xml_file_counter = None
no_record_counter = None
no_object_type_counter = None
no_object_type_text_counter = None

# init
object_type_to_count_map = {}

# get file list.
xml_file_list = glob.glob( "{}/*.xml".format( uncompressed_archive_path ) )

# loop
xml_file_counter = 0
no_record_counter = 0
no_object_type_counter = 0
no_object_type_text_counter = 0
for xml_file_path in xml_file_list:
    
    xml_file_counter += 1
    
    # try to parse the file
    with open( xml_file_path ) as xml_file:
    
        # parse XML
        xml_dict = xmltodict.parse( xml_file.read() )
        
        # get root.Record.ObjectType value
        record_node = xml_dict.get( "Record", None )
        
        if ( record_node is not None ):
            
            # get object type (looks like xmltodict stores
            #     elements with no attributes and no child
            #     elements just as a string contents mapped
            #     to name in parent, no dictionary)
            #
            # so for:
            # <Record>
            #     ...
            #     <ObjectType>Advertisement</ObjectType>
            #     ...
            # </Record>
            #
            # to get value:
            #     record_node = xml_dict.get( "Record", None )
            #     object_type_list = record_node.get( "ObjectType", None )
            #     object_type = "|".join( object_type_list )
            #
            # NOT:
            #     record_node = xml_dict.get( "Record", None )
            #     object_type_node = record_node.get( "ObjectType", None )
            #     object_type = object_type_node.get( "#text", None )
            #
            # Doc that led me astray: https://docs.python-guide.org/scenarios/xml/
            object_type_list = record_node.get( "ObjectType", None )
            object_type = "|".join( object_type_list )

            # got a type?
            if ( ( object_type is not None ) and ( object_type != "" ) ):

                # we do.  Increment count.
                object_type_count = object_type_to_count_map.get( object_type, 0 )
                object_type_count += 1
                object_type_to_count_map[ object_type ] = object_type_count

            else:

                # object type is None
                no_object_type_text_counter += 1

            #-- END check for type value --#
            
        else:
            
            # increment counter
            no_record_counter += 1
            
        #-- END check if we found a "Record" node in root --#

    #-- END with open( xml_file_path )...: --#
    
#-- END loop over XML files --#

print( "XML file count: {}".format( len( xml_file_list ) ) )
print( "Counters:" )
print( "- Processed {} files".format( xml_file_counter ) )
print( "- No Record: {}".format( no_record_counter ) )
print( "- No ObjectType: {}".format( no_object_type_counter ) )
print( "- No ObjectType value: {}".format( no_object_type_text_counter ) )
print( "\nObjectType values and occurrence counts:")
for object_type, object_type_count in six.iteritems( object_type_to_count_map ):
    
    # print type and count
    print( "- {}: {}".format( object_type, object_type_count ) )
    
#-- END loop over object types. --#

## build list of all ObjectTypes

- Back to [Table of Contents](#Table-of-Contents)

Loop over all folders in the paper path.  For each folder, grab all files in the folder.  For each file, parse XML, then get the ObjectType value and if it isn't already in map of obect types to counts, add it.  Increment count.

In [9]:
xml_folder_list = glob.glob( "{}/*".format( uncompressed_paper_path ) )
log_message = "folder_list: {}".format( xml_folder_list )
my_logging_helper.output_debug_message( log_message, do_print_IN = True )


folder_list: ['/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413220638_00001', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413222205_00001', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413222207_00002', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413222309_00003', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413222310_00004', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413222312_00005', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413222313_00006', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413222414_00007', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413222416_00008', '/mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM

In [10]:
# declare variables
xml_file_list = None
xml_file_path = None
xml_file = None
xml_dict = None
xml_file_counter = None
object_type_to_count_map = None
object_type_count = None
record_node = None
object_type_node = None
object_type_list = None
object_type = None

# declare variables - folders
xml_folder_list = None
xml_folder_path = None
xml_folder_count = None
xml_folder_counter = None
xml_folder_start_time = None
xml_folder_end_time = None
xml_folder_duration = None

# declare variables - auditing
total_file_count = None
xml_file_count = None
xml_file_counter = None
no_record_counter = None
no_object_type_counter = None
no_object_type_text_counter = None

# init
object_type_to_count_map = {}

# first, get the list of folders for the current paper.
xml_folder_list = glob.glob( "{}/*".format( uncompressed_paper_path ) )
xml_folder_count = len( xml_folder_list )

log_message = "Processing {} XML folders in {}".format( xml_folder_count, uncompressed_paper_path )
my_logging_helper.output_debug_message( log_message, do_print_IN = True )

# loop over the folders
xml_folder_counter = 0
total_file_count = 0
for xml_folder_path in xml_folder_list:
    
    xml_folder_counter += 1
    
    # log the folder
    xml_folder_start_time = datetime.datetime.now()
    log_message = "==> Processing XML folder {} ( {} of {} ) @ {}".format( xml_folder_path, xml_folder_counter, xml_folder_count, xml_folder_start_time )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )

    # get file list.
    xml_file_list = glob.glob( "{}/*.xml".format( xml_folder_path ) )
    xml_file_count = len( xml_file_list )
    total_file_count += xml_file_count

    # log the count
    log_message = "----> XML file count: {}".format( xml_file_count )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )

    # loop
    xml_file_counter = 0
    no_record_counter = 0
    no_object_type_counter = 0
    no_object_type_text_counter = 0
    for xml_file_path in xml_file_list:

        xml_file_counter += 1

        # try to parse the file
        with open( xml_file_path ) as xml_file:

            # parse XML
            xml_dict = xmltodict.parse( xml_file.read() )

            # get root.Record.ObjectType value
            record_node = xml_dict.get( "Record", None )

            if ( record_node is not None ):

                # get object type (looks like xmltodict stores
                #     elements with no attributes and no child
                #     elements just as a string contents mapped
                #     to name in parent, no dictionary)
                #
                # so for:
                # <Record>
                #     ...
                #     <ObjectType>Advertisement</ObjectType>
                #     ...
                # </Record>
                #
                # to get value:
                #     record_node = xml_dict.get( "Record", None )
                #     object_type_list = record_node.get( "ObjectType", None )
                #     object_type = "|".join( object_type_list )
                #
                # NOT:
                #     record_node = xml_dict.get( "Record", None )
                #     object_type_node = record_node.get( "ObjectType", None )
                #     object_type = object_type_node.get( "#text", None )
                #
                # Doc that led me astray: https://docs.python-guide.org/scenarios/xml/
                object_type_list = record_node.get( "ObjectType", None )
                object_type = "|".join( object_type_list )

                # got a type?
                if ( ( object_type is not None ) and ( object_type != "" ) ):

                    # we do.  Increment count.
                    object_type_count = object_type_to_count_map.get( object_type, 0 )
                    object_type_count += 1
                    object_type_to_count_map[ object_type ] = object_type_count

                else:

                    # object type is None
                    no_object_type_text_counter += 1

                #-- END check for type value --#

            else:

                # increment counter
                no_record_counter += 1

            #-- END check if we found a "Record" node in root --#

        #-- END with open( xml_file_path )...: --#

    #-- END loop over XML files --#
    
    # log the folder
    xml_folder_end_time = datetime.datetime.now()
    xml_folder_duration = xml_folder_end_time - xml_folder_start_time
    log_message = "----> Processing complete @ {} ( duration {} )\n".format( xml_folder_end_time, xml_folder_duration )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )

#-- END loop over XML directories --#

log_message = "Counters:"
my_logging_helper.output_debug_message( log_message, do_print_IN = True )
log_message = "- Processed {} files".format( total_file_count )
my_logging_helper.output_debug_message( log_message, do_print_IN = True )
log_message = "- No Record: {}".format( no_record_counter )
my_logging_helper.output_debug_message( log_message, do_print_IN = True )
log_message = "- No ObjectType: {}".format( no_object_type_counter )
my_logging_helper.output_debug_message( log_message, do_print_IN = True )
log_message = "- No ObjectType value: {}".format( no_object_type_text_counter )
my_logging_helper.output_debug_message( log_message, do_print_IN = True )
log_message = "\nObjectType values and occurrence counts:"
my_logging_helper.output_debug_message( log_message, do_print_IN = True )

for object_type, object_type_count in six.iteritems( object_type_to_count_map ):
    
    # print type and count
    log_message = "- {}: {}".format( object_type, object_type_count )
    my_logging_helper.output_debug_message( log_message, do_print_IN = True )

#-- END loop over object types. --#

Processing 263 XML folders in /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor
==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413220638_00001 ( 1 of 263 ) @ 2019-08-10 02:04:06.200530
----> XML file count: 22847
----> Processing complete @ 2019-08-10 02:05:25.418955 ( duration 0:01:19.218425 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413222205_00001 ( 2 of 263 ) @ 2019-08-10 02:05:25.419820
----> XML file count: 25000
----> Processing complete @ 2019-08-10 02:06:51.088789 ( duration 0:01:25.668969 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413222207_00002 ( 3 of 263 ) @ 2019-08-10 02:06:51.089166
----> XML file count: 25000
----> Processing complete @ 2019-08-10 02:08:17.001151 ( duration 0:01:25.911985 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/u

----> XML file count: 25000
----> Processing complete @ 2019-08-10 02:46:44.686613 ( duration 0:01:23.999485 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413223956_00007 ( 31 of 263 ) @ 2019-08-10 02:46:44.686983
----> XML file count: 25000
----> Processing complete @ 2019-08-10 02:48:07.687867 ( duration 0:01:23.000884 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413224001_00010 ( 32 of 263 ) @ 2019-08-10 02:48:07.688274
----> XML file count: 25000
----> Processing complete @ 2019-08-10 02:49:34.069049 ( duration 0:01:26.380775 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413224057_00008 ( 33 of 263 ) @ 2019-08-10 02:49:34.069422
----> XML file count: 25000
----> Processing complete @ 2019-08-10 02:50:58.399953 ( duration 0:01:24.330531 )

==> Processing XML folder /mnt/hgfs/projects/phd/pr

----> XML file count: 25000
----> Processing complete @ 2019-08-10 03:29:11.288925 ( duration 0:01:25.047589 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413224836_00037 ( 61 of 263 ) @ 2019-08-10 03:29:11.289348
----> XML file count: 25000
----> Processing complete @ 2019-08-10 03:30:36.523647 ( duration 0:01:25.234299 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413224939_00038 ( 62 of 263 ) @ 2019-08-10 03:30:36.524021
----> XML file count: 25000
----> Processing complete @ 2019-08-10 03:32:02.233027 ( duration 0:01:25.709006 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413224940_00039 ( 63 of 263 ) @ 2019-08-10 03:32:02.233436
----> XML file count: 25000
----> Processing complete @ 2019-08-10 03:33:26.084228 ( duration 0:01:23.850792 )

==> Processing XML folder /mnt/hgfs/projects/phd/pr

----> XML file count: 25000
----> Processing complete @ 2019-08-10 04:10:51.307596 ( duration 0:01:23.689559 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413225921_00023 ( 91 of 263 ) @ 2019-08-10 04:10:51.308316
----> XML file count: 25000
----> Processing complete @ 2019-08-10 04:12:13.344772 ( duration 0:01:22.036456 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413225922_00024 ( 92 of 263 ) @ 2019-08-10 04:12:13.345588
----> XML file count: 25000
----> Processing complete @ 2019-08-10 04:13:36.689555 ( duration 0:01:23.343967 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413230023_00025 ( 93 of 263 ) @ 2019-08-10 04:13:36.690260
----> XML file count: 25000
----> Processing complete @ 2019-08-10 04:14:58.884002 ( duration 0:01:22.193742 )

==> Processing XML folder /mnt/hgfs/projects/phd/pr

----> XML file count: 25000
----> Processing complete @ 2019-08-10 04:51:44.606077 ( duration 0:01:22.830411 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413231006_00006 ( 121 of 263 ) @ 2019-08-10 04:51:44.606446
----> XML file count: 25000
----> Processing complete @ 2019-08-10 04:53:06.785859 ( duration 0:01:22.179413 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413231007_00007 ( 122 of 263 ) @ 2019-08-10 04:53:06.786190
----> XML file count: 25000
----> Processing complete @ 2019-08-10 04:54:31.803470 ( duration 0:01:25.017280 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413231009_00008 ( 123 of 263 ) @ 2019-08-10 04:54:31.804183
----> XML file count: 25000
----> Processing complete @ 2019-08-10 04:55:56.719680 ( duration 0:01:24.915497 )

==> Processing XML folder /mnt/hgfs/projects/phd

----> XML file count: 25000
----> Processing complete @ 2019-08-10 05:33:35.048229 ( duration 0:01:22.949376 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413231946_00036 ( 151 of 263 ) @ 2019-08-10 05:33:35.048590
----> XML file count: 25000
----> Processing complete @ 2019-08-10 05:34:54.474312 ( duration 0:01:19.425722 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413232047_00037 ( 152 of 263 ) @ 2019-08-10 05:34:54.474987
----> XML file count: 25000
----> Processing complete @ 2019-08-10 05:36:15.488709 ( duration 0:01:21.013722 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413232048_00038 ( 153 of 263 ) @ 2019-08-10 05:36:15.489067
----> XML file count: 25000
----> Processing complete @ 2019-08-10 05:37:37.007694 ( duration 0:01:21.518627 )

==> Processing XML folder /mnt/hgfs/projects/phd

----> XML file count: 25000
----> Processing complete @ 2019-08-10 06:15:04.149695 ( duration 0:01:24.457617 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413232825_00019 ( 181 of 263 ) @ 2019-08-10 06:15:04.150031
----> XML file count: 25000
----> Processing complete @ 2019-08-10 06:16:28.848668 ( duration 0:01:24.698637 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413232926_00020 ( 182 of 263 ) @ 2019-08-10 06:16:28.853694
----> XML file count: 25000
----> Processing complete @ 2019-08-10 06:17:52.403996 ( duration 0:01:23.550302 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413232927_00021 ( 183 of 263 ) @ 2019-08-10 06:17:52.404683
----> XML file count: 25000
----> Processing complete @ 2019-08-10 06:19:16.050375 ( duration 0:01:23.645692 )

==> Processing XML folder /mnt/hgfs/projects/phd

----> XML file count: 25000
----> Processing complete @ 2019-08-10 07:39:31.411865 ( duration 0:01:23.187899 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413233803_00002 ( 211 of 263 ) @ 2019-08-10 07:39:31.412242
----> XML file count: 25000
----> Processing complete @ 2019-08-10 07:40:53.414766 ( duration 0:01:22.002524 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413233904_00003 ( 212 of 263 ) @ 2019-08-10 07:40:53.415214
----> XML file count: 25000
----> Processing complete @ 2019-08-10 07:42:15.976946 ( duration 0:01:22.561732 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413233906_00004 ( 213 of 263 ) @ 2019-08-10 07:42:15.977308
----> XML file count: 25000
----> Processing complete @ 2019-08-10 07:43:38.460584 ( duration 0:01:22.483276 )

==> Processing XML folder /mnt/hgfs/projects/phd

----> XML file count: 25000
----> Processing complete @ 2019-08-10 08:20:55.650795 ( duration 0:01:23.016310 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413234741_00032 ( 241 of 263 ) @ 2019-08-10 08:20:55.651534
----> XML file count: 25000
----> Processing complete @ 2019-08-10 08:22:20.050541 ( duration 0:01:24.399007 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413234842_00033 ( 242 of 263 ) @ 2019-08-10 08:22:20.051339
----> XML file count: 25000
----> Processing complete @ 2019-08-10 08:23:43.124435 ( duration 0:01:23.073096 )

==> Processing XML folder /mnt/hgfs/projects/phd/proquest_hnp/uncompressed/ChristianScienceMonitor/CSM_20170413234843_00034 ( 243 of 263 ) @ 2019-08-10 08:23:43.124885
----> XML file count: 25000
----> Processing complete @ 2019-08-10 08:25:05.943246 ( duration 0:01:22.818361 )

==> Processing XML folder /mnt/hgfs/projects/phd

# TODO

- Back to [Table of Contents](#Table-of-Contents)

TODO:

- create bash script to uncompress all files in a folder from data to uncompressed, when passed paper and archive identifiers?